In [1]:
import autograd.numpy as np
from autograd import grad, hessian, jacobian
from scipy.optimize import minimize

In [2]:
import matplotlib.pyplot as plt
from counterfactual_functions import *
from helper import *

username='boraozaltun'

data_dict = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/final_data/data_subset_20200304.pickle')

reg_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/reg_2_num.pickle')
comm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/comm_2_num.pickle')

n = data_dict['n']
g = data_dict['g']
k = data_dict['k']

data_dict['R_hat'] = np.ones((n, g))
data_dict['R_hat'][reg_2_num['usa'], 3] = 2.
data_dict['R_hat'][reg_2_num['usa'], 4] = 2.



X_0 = np.ones(n*g + n)*0.5


eval_F = lambda x: reduced_counterfactual_overdetermined(x, data_dict)
# eval_f = lambda x: np.sum(eval_F(x)**2)
eval_f = lambda x: np.linalg.norm(eval_F(x))

eval_grad_f = grad(eval_f)
eval_hess_f = hessian(eval_f)
bounds_f = [(1e-14, None) for x in range(n*g+n)]

In [3]:
sol = minimize(eval_f, X_0*2, jac=eval_grad_f, hess=eval_hess_f, method='trust-krylov', bounds=bounds_f)#,

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/autograd/numpy/numpy_vjps.py:444: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return lambda g: g[idxs]
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/autograd/numpy/numpy_boxes.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  def __getitem__(A, idx): return A[idx]
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/autograd/numpy/numpy_vjps.py:597: FutureWarning: Using a non-tuple sequence for multid

In [4]:
sol.x

array([0.99638288, 1.00311148, 1.00233349, 0.90787821, 0.92446246,
       0.99695359, 0.9937006 , 1.00042825, 1.00250847, 0.91174359,
       0.99815276, 0.98947113, 0.99686863, 0.99992468, 1.00174645,
       0.91310301, 0.85180209, 0.99621225, 1.00273   , 1.0046263 ,
       1.00205469, 0.89398613, 0.95554409, 1.00671464, 0.99446112,
       1.0049035 , 0.99905193, 0.96681557, 0.90370577, 0.97619898,
       1.00850613, 1.00859533, 1.0068069 , 0.88409406, 0.89992938,
       1.00737273, 0.99853655, 1.00098347, 1.00210184, 0.91387955,
       0.93801526, 0.99356991, 0.97035654, 0.98990349, 0.99640583,
       0.92073355, 0.87491546, 0.99646808, 0.9977464 , 0.99825351,
       1.00496032, 0.86742333, 0.9320074 , 0.99086338, 0.98472334,
       0.99527859, 0.98999841, 0.90248652, 0.96445793, 0.99408344,
       0.96651795, 0.99500502, 0.9991635 , 0.89634016, 0.87738548,
       0.96360339, 0.99722252, 1.00050113, 1.00073584, 0.91397589,
       0.93189488, 0.9972811 , 0.98665207, 0.98260364, 0.99581

In [5]:
sol.success

True

In [6]:
sol

     fun: 0.015919534758830498
    hess: array([[ 5.34111904e+01,  8.09374241e-02,  1.86843921e-01, ...,
         3.88163112e+00, -2.01524833e-01, -4.33950343e-01],
       [ 8.09374241e-02,  4.07534981e+01, -1.82702113e-01, ...,
         2.17047111e+00, -7.47326895e+00, -8.04239325e-01],
       [ 1.86843921e-01, -1.82702113e-01,  3.23443957e+03, ...,
         2.45558579e+02, -1.23601870e+01, -3.69983285e+00],
       ...,
       [ 3.88163112e+00,  2.17047111e+00,  2.45558579e+02, ...,
         4.23276857e+03, -2.53157458e+02, -1.24427551e+01],
       [-2.01524833e-01, -7.47326895e+00, -1.23601870e+01, ...,
        -2.53157458e+02,  2.85646136e+03, -2.28451446e-01],
       [-4.33950343e-01, -8.04239325e-01, -3.69983285e+00, ...,
        -1.24427551e+01, -2.28451446e-01,  1.46940843e+03]])
     jac: array([-2.71802799e-09,  5.91297140e-10, -2.64283448e-08, -8.50569522e-09,
        1.09023495e-08,  2.88001956e-09,  1.08630183e-10,  6.01980017e-11,
        6.67375634e-10,  5.26618997e-10,  

In [7]:
## Seems like constraining the length of w_hat and r_hat works! Try to justify this later!

In [8]:
r_hat_star, w_hat_star = get_values_from_X_reduced(sol.x, data_dict)

In [9]:
E_hat_star = get_E_hat(w_hat_star, r_hat_star, data_dict)

In [10]:
E_hat_star.sum()

132.97258996753982